# Modelling

## Imports

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential, layers, models
from sklearn.preprocessing import MinMaxScaler, RobustScaler

2023-06-12 10:01:07.755653: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 10:01:08.054296: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-12 10:01:08.958231: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-12 10:01:08.958304: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

## y_train and y_test

In [3]:
y_dataframe = pd.read_csv('../data/y_dataframe.csv', index_col=0)

/tmp/ipykernel_21571/3703226929.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  y_dataframe = pd.read_csv('../data/y_dataframe.csv', index_col=0)


In [4]:
y_dataframe

,1950s,1970,1971,2009,4-way,abandon,about,above,abraham,abstract,...,yogurt,york,you,young,younger,youth,zebra,zip,zombie,zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X_test_id_977,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X_test_id_978,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X_test_id_979,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X_test_id_980,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
y_train = y_dataframe.iloc[0:8859].to_numpy()

In [6]:
y_test = y_dataframe.iloc[8859:9842].to_numpy()

In [7]:
y_train.shape

(8859, 2637)

In [8]:
y_test.shape

(982, 2637)

## X_train and X_test

In [8]:
X_train = np.load('../raw_data/nsd_train_fmriavg_nsdgeneral_sub1.npy')

In [9]:
X_test = np.load('../raw_data/nsd_test_fmriavg_nsdgeneral_sub1.npy')

In [10]:
X_train.shape

(8859, 15724)

In [11]:
X_test.shape

(982, 15724)

In [12]:
scaler = RobustScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Modelling

In [16]:

model = models.Sequential()

model = Sequential([layers.Dense(100, input_shape = (15724,), activation = "relu"),
                  layers.Dense(100, activation = "relu"),
                    layers.Dropout(0.4),
                  layers.Dense(200, activation = "relu"),
                    layers.Dropout(0.4),
                  layers.Dense(500, activation = "relu"),
                    layers.Dropout(0.4),
                    layers.Dense(500, activation = "relu"),
                    layers.Dropout(0.4),
                    layers.Dense(1000, activation = "relu"),
                    layers.Dropout(0.4),
                    layers.Dense(2000, activation = "relu"),
                  layers.Dense(2637, activation = "sigmoid")])


# model.add(layers.Dense(2637, activation='sigmoid'))

In [17]:
model.compile(optimizer='adam',
    loss='binary_crossentropy',
             metrics = ['accuracy'])

In [18]:
history = model.fit(X_train_scaled,
                    y_train, 
                    validation_split = 0.1,
                    batch_size = 32,
                    epochs=30)

Epoch 1/30
250/250 [==============================] - 5s 19ms/step - loss: 0.0203 - accuracy: 0.0206 - val_loss: 0.0368 - val_accuracy: 0.0056
Epoch 2/30
250/250 [==============================] - 4s 17ms/step - loss: 0.0095 - accuracy: 0.0251 - val_loss: 0.0142 - val_accuracy: 0.0248
Epoch 3/30
250/250 [==============================] - 4s 17ms/step - loss: 0.0088 - accuracy: 0.0236 - val_loss: 0.0086 - val_accuracy: 0.0271
Epoch 4/30
250/250 [==============================] - 4s 17ms/step - loss: 0.0085 - accuracy: 0.0235 - val_loss: 0.0085 - val_accuracy: 0.0293
Epoch 5/30
250/250 [==============================] - 4s 17ms/step - loss: 0.0084 - accuracy: 0.0252 - val_loss: 0.0084 - val_accuracy: 0.0327
Epoch 6/30
250/250 [==============================] - 4s 18ms/step - loss: 0.0083 - accuracy: 0.0263 - val_loss: 0.0083 - val_accuracy: 0.0327
Epoch 7/30
250/250 [==============================] - 4s 18ms/step - loss: 0.0081 - accuracy: 0.0275 - val_loss: 0.0082 - val_accuracy: 0.0350

In [19]:
answer = model.predict(X_train_scaled[10].reshape(1,-1))

1/1 [==============================] - 0s 75ms/step


In [20]:
answer[answer>0.1]

array([0.14293751, 0.60508615, 0.20769837, 0.13430378, 0.12713829],
      dtype=float32)

In [21]:
answer_idxs = np.argpartition(answer[0],-4)[-4:]

In [22]:
answer_idxs

array([2064,  383,  703,  187])

In [40]:
y_dataframe

,1950s,1970,1971,2009,4-way,abandon,about,above,abraham,abstract,...,yogurt,york,you,young,younger,youth,zebra,zip,zombie,zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X_test_id_977,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X_test_id_978,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X_test_id_979,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X_test_id_980,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
labels = y_dataframe.columns[1:]

In [24]:
labels[answer_idxs]

Index(['site', 'catch', 'doll', 'bedlamps'], dtype='object')

In [35]:
test_answer = model.predict(X_test_scaled[45].reshape(1,-1))

1/1 [==============================] - 0s 13ms/step


In [36]:
test_answer

array([[8.0414657e-06, 1.6014625e-05, 2.3940924e-05, ..., 2.3906901e-03,
        4.0527619e-05, 9.6112315e-05]], dtype=float32)

In [37]:
test_answer_idxs = np.argpartition(test_answer[0],-4)[-4:]

In [38]:
test_answer_idxs

array([ 149, 2297, 2497,  885])

In [39]:
labels[test_answer_idxs]

Index(['bank', 'tablecloth', 'vast', 'flowery'], dtype='object')